In [ ]:
# import Pkg; Pkg.add("Plots")

In [ ]:
using DifferentialEquations, Plots

In [ ]:
function I_eff(I, Aβ, τ, K_R, K_τ)
    """ Multiplicative competitive inhibition.

    Masking of insulin receptors by Aβ & tau.
    Aβ oligomers block IR binding; p-tau blocks downstream signalling.
    Uses two independent competitive terms multiplied (equivalent to adding free energies of inhibition).
    """
    inhibitors = (1 + (Aβ / K_R)) * (1 + (τ / K_τ))
    return I / inhibitors 
end

In [ ]:
function V_ide(I, V_0, K_ρ, ρ)
    """ Michaelis-Menten-like regulatory rise.

    Insulin-dependent IDE induction.
    The maximum catalytic capacity of IDE increases with insulin signalling until a new plateau (V0(1+ρ)V0​(1+ρ)).
    Standard hyperbolic activation; no change.
    """
    return V_0 * (1 + ρ * (I / (K_ρ + I)))
end

In [ ]:
function rm3!(du, u, p, t)
    I, Aβ, τ = u
    S_0, S_Aβ_0, K_BBB, K_Aβ, K_I, K_ρ, K_R, K_τ, k_clr, k_phos, k_dephos, S_Aβ, I_50, σ, μ, γ, ρ, n = p

    """
    first term: Insulin transport brake. Michaelis-Menten inhibition of a source term.
    - Brain uptake of plasma insulin slows as interstitial insulin climbs (saturable BBB transporter).
    - Same algebraic structure as MM, but applied to a supply flux rather than a clearance flux.

    second term: Linear (first-order) clearance.
    - Non-IDE clearance routes.

    third term: IDE-mediated clearance. Competitive Michaelis-Menten.
    - Insulin is removed by insulin-degrading enzyme at a rate that saturates with its own
    concentration and is competitively slowed by the other substrate.
    - Only change from the classic form is that Vmax⁡ itself is allowed to depend on II (see next row).
    """
    I_dot = (S_0 / (1 + (I / K_BBB))) - k_clr*I - (V_ide(I, V_0, K_ρ, ρ)*I / K_I*(1 + (Aβ / K_Aβ) + I))

    """
    first term: basal influx minus direct anti-amyloid action of insulin. Linear mass-action.
    - Higher insulin pushes APP down the non-amyloidogenic route, lowering Aβ production.
    - No novelty; included because review 3 explicitly documents the effect.

    second term: Linear (first-order) clearance.
    - Non-IDE clearance routes.

    third term: IDE-mediated clearance. Competitive Michaelis-Menten.
    - Aβ is removed by insulin-degrading enzyme at a rate that saturates with its own
    concentration and is competitively slowed by the other substrate.
    - Only change from the classic form is that Vmax⁡ itself is allowed to depend on II (see next row).

    fourth term: IDE crowding by insulin in Aβ branch. Competitive modifier grafted onto the Aβ MM term.
    - When insulin hogs IDE, Aβ clearance is further slowed.
    - Minor deviation: instead of placing II in both the numerator & denominator, we simply multiply by
    the fractional unoccupied IDE.
    """
    Aβ_dot = (S_Aβ_0 - σ*I) - μ*Aβ - (V_ide(I, V_0, K_ρ, ρ)*Aβ / (K_Aβ + Aβ)) * (1 / (1 + (I / K_I)))

    """
    first term: Kinase inhibition by insulin. Hill function (sigmoidal repression).
    - Active glycogen-synthase-kinase-3β (and JNK) fraction falls as functional insulin rises.
    - Exactly the textbook Hill form; nn tunes ultrasensitivity.

    second term: Aβ-driven tau‐P surge. Linear drive.
    - Oligomeric Aβ elevates tau phosphorylation via TNFα–JNK → IRS-1 blockade.
    - Kept linear: reviews give no evidence for saturation within physiological Aβ range.

    third term: Linear (first-order) clearance.
    - Constitutive dephosphorylation
    
    """
    τ_dot = k_phos*(1 / (1 + (I_eff(I, Aβ, τ, K_R, K_τ) / I_50)^n)) + γ*Aβ - k_dephos*τ

    du[1] = I_dot
    du[2] = Aβ_dot
    du[3] = τ_dot
end

In [ ]:
u0 = [1.0; 0.5; 0.5]
tspan = (0.0, 1000.0)

"""units
T: hour (h)
C: micromole (μM * Liter)
"""

""" basal values, anchor linear supply processes.
Zero-order synthesis / influx rates (fluxes) independent of the state variables.

They feed material into the system. We kept the S-notation to flag parameters you’d
normally get from uptake or secretion assays rather than from binding kinetics.

concentration · time−1−1 (e.g., μM h−1−1)
"""
S_0 = 0.12        # μM⋅h^{-1] Maximal plasma‐to‑brain insulin flux (post‑prandial peak). 2/4
S_Aβ_0 = 0.005    # μM⋅h^{-1} −1	Neuronal APP shedding gives ~0.5 pmol h−1−1 mg−1−1 protein; scaled to tissue. 2/4

# basal max IDE rate
V_0 = 5.0         # μM⋅h^{-1} Microdialysis turnover fit 2/4

""" Affinity/threshold constants in denominator-type terms (MM, Hill, competition), anchor non-linear saturation or competition.

Numerically they are points at which the denominator doubles.
In enzymology & regulatory kinetics, capital KK traditionally marks a binding constant or a half-effect concentration.

Same as species they relate to (μM, nM…)
"""
K_BBB = 0.5       # μM Saturation constant for insulin transporter; set near upper CSF insulin during euglycaemic clamp. 2/4
K_Aβ = 0.3        # μM Same paper, Aβ as substrate, IDE affinity for Aβ 3/4
K_I = 1.0         # μM IDE kinetics (rat brain cytosol), IDE affinity for insulin 3/4
K_ρ = 0.50        # μM IDE transcription half‑max by insulin (hepatocyte & neuron culture). 2/4
K_R = 0.10        # μM Aβ dimer concentration giving 50 % IR inhibition (binding assays). 2/4
K_τ = 0.05        # p‑tau concentration giving 50 % IRS‑1 blockade (cell culture). 2/4

# rate constants
k_clr = 0.20      # h^{-1} Brain interstitial insulin T1/2 ≈ 3.5 h ⇒ k = ln 2 / 3.5 h. 3/4
k_phos = 0.05     # h^{-1} In-vitro tau-P time course 3/4
k_dephos = 0.02   # h^{-1} PP2A activity assays 3/4

""" half-maximal inhibitory concentration of functional insulin for the kinase block that limits tau phosphorylation.

Mathematically it is the value of IeffIeff​ at which the Hill factor drops to 1/2.
Biologically it aggregates two empirical numbers measured in the literature:
- the concentration needed to activate Akt strongly enough to inhibit GSK-3β, plus
- the concentration that restrains JNK via IRS-1 serine de-phosphorylation.
"""
I_50 = 0.0002     # 0.2 nM, 50 % Akt/GSK‑3β block 3/4

σ = 0.010         # h^{-1} Insulin‑driven shift to non‑amyloidogenic APP (10 % of baseline Aβ generation per µM insulin). 2/4
μ = 0.010         # h^{-1} Bulk Aβ proteolysis / drainage half‑life ≈ 70 h. 3/4
γ = 0.050         # h^{-1} Strength of Aβ‑induced tau phosphorylation (p‑tau doubling in ≈ 14 h when Aβ ≈ 1 µM). 2/4
ρ = 1.0           # PI3K-IDE induction assays, IDE Vmax⁡ doubling 2/4

# hill factor
n = 3.0           # Typical GSK-3β ultrasensitivity, hill coefficient, 3/4

p = [S_0; S_Aβ_0; V_0; K_BBB; K_Aβ; K_I; K_ρ; K_R; K_τ; k_clr; k_phos; k_dephos; I_50; σ; μ; γ; ρ; n]

In [ ]:
prob = ODEProblem(rm3!, u0, tspan, p)
sol = solve(prob)

In [ ]:
plot(sol)